In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
### Read the dataset
Data = pd.read_csv("clean_data_after_eda.csv")
Data['date_activ'] = pd.to_datetime(Data['date_activ'], format='%Y-%m-%d')
Data['date_end'] = pd.to_datetime(Data['date_end'], format='%Y-%m-%d')
Data['date_modif_prod'] = pd.to_datetime(Data['date_modif_prod'], format='%Y-%m-%d')
Data['date_renewal'] = pd.to_datetime(Data['date_renewal'], format='%Y-%m-%d')

Data.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.000131,4.100838e-05,0.000908,2.086294,99.530517,44.235794,2.086425,9.953056e+01,44.236702,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000003,1.217891e-03,0.000000,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000004,9.450150e-08,0.000000,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000,0


In [3]:
list(Data)

['id',
 'channel_sales',
 'cons_12m',
 'cons_gas_12m',
 'cons_last_month',
 'date_activ',
 'date_end',
 'date_modif_prod',
 'date_renewal',
 'forecast_cons_12m',
 'forecast_cons_year',
 'forecast_discount_energy',
 'forecast_meter_rent_12m',
 'forecast_price_energy_off_peak',
 'forecast_price_energy_peak',
 'forecast_price_pow_off_peak',
 'has_gas',
 'imp_cons',
 'margin_gross_pow_ele',
 'margin_net_pow_ele',
 'nb_prod_act',
 'net_margin',
 'num_years_antig',
 'origin_up',
 'pow_max',
 'var_year_price_off_peak_var',
 'var_year_price_peak_var',
 'var_year_price_mid_peak_var',
 'var_year_price_off_peak_fix',
 'var_year_price_peak_fix',
 'var_year_price_mid_peak_fix',
 'var_year_price_off_peak',
 'var_year_price_peak',
 'var_year_price_mid_peak',
 'var_6m_price_off_peak_var',
 'var_6m_price_peak_var',
 'var_6m_price_mid_peak_var',
 'var_6m_price_off_peak_fix',
 'var_6m_price_peak_fix',
 'var_6m_price_mid_peak_fix',
 'var_6m_price_off_peak',
 'var_6m_price_peak',
 'var_6m_price_mid_peak',


### Feature Engineering.

### Difference between Off-peak prices in December and the next January.

In [4]:
price_df = pd.read_csv('price_data (1).csv')
price_df['price_date'] = pd.to_datetime(price_df['price_date'], format = '%Y-%m-%d')
price_df.tail(3)

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
192999,16f51cdc2baa19af0b940ee1b3dd17d5,2015-10-01,0.119916,0.102232,0.076257,40.728885,24.43733,16.291555
193000,16f51cdc2baa19af0b940ee1b3dd17d5,2015-11-01,0.119916,0.102232,0.076257,40.728885,24.43733,16.291555
193001,16f51cdc2baa19af0b940ee1b3dd17d5,2015-12-01,0.119916,0.102232,0.076257,40.728885,24.43733,16.291555


In [5]:
## Get Months Column from the Dataset

price_df['month'] = price_df['price_date'].dt.month

In [6]:
## Get Jan and Dec 

jan_price = price_df[price_df['month'] == 1].groupby('id').agg({'price_off_peak_var':'mean', 'price_off_peak_fix':'mean'}).reset_index()
dec_price = price_df[price_df['month'] == 12].groupby('id').agg({'price_off_peak_var':'mean', 'price_off_peak_fix':'mean'}).reset_index()


In [7]:
### Perform the differences.


diff = pd.merge(dec_price.rename(columns={'price_off_peak_var':'dec_1', 'price_off_peak_fix':'dec_2'}), jan_price, on='id')

diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']
diff = diff[['id','offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]
diff



### Do a Merge to the datasets.

Data = pd.merge(Data,diff, on='id')

### Difference between Peak Prices in January and Dec.

In [8]:
price_df.tail(3)


## Get the jan and dec of Peak prices

jan_peak = price_df[price_df['month'] ==1 ].groupby('id').agg({'price_peak_var':'mean', 'price_peak_fix':'mean'}).reset_index()
dec_peak = price_df[price_df['month']==12].groupby('id').agg({'price_peak_var':'mean', 'price_peak_fix':'mean'}).reset_index()

In [9]:
## Perform the difference

diff_peak = pd.merge(dec_peak.rename(columns={'price_peak_var':'dec_1', 'price_peak_fix':'dec_2'}),jan_peak, on = 'id')


diff_peak['peak_diff_dec_jan_energy'] = diff_peak['dec_1'] - diff_peak['price_peak_var']
diff_peak['peak_diff_dec_jan_power'] = diff_peak['dec_2'] - diff_peak['price_peak_var']

### Merge the differences back to the price_df


temp_df = pd.merge(price_df, diff, on='id')
new_price_df = pd.merge(temp_df,diff_peak, on ='id')


diff_peak = diff_peak[['id','peak_diff_dec_jan_energy','peak_diff_dec_jan_power']]
diff_peak

,id,peak_diff_dec_jan_energy,peak_diff_dec_jan_power
0,0002203ffbb812588b632b9e628cc38d,-0.002302,24.333355
1,0004351ebdd665e6ee664792efc4fd13,0.000000,0.000000
2,0010bcc39e42b3c2131ed2ce55246e3c,0.000000,0.000000
3,0010ee3855fdea87602a5b7aba8e42de,-0.005120,24.336825
4,00114d74e963e47177db89bc70108537,0.000000,0.000000
...,...,...,...
16063,ffef185810e44254c3a4c6395e6b4d8a,-0.038788,24.303738
16064,fffac626da707b1b5ab11e8431a4d0a2,0.000000,0.000000
16065,fffc0cacd305dd51f316424bbb08d1bd,-0.003707,24.806752
16066,fffe4f5646aa39c7f97f95ae2679ce64,-0.004937,24.331902


In [10]:
### Merge the differences back to the price_df


temp_df = pd.merge(price_df, diff, on='id')
new_price_df = pd.merge(temp_df,diff_peak, on ='id')

In [11]:
temp_df.head(5)

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix,month,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0,1,-0.005508,0.177779
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0,2,-0.005508,0.177779
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0,3,-0.005508,0.177779
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0,4,-0.005508,0.177779
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0,5,-0.005508,0.177779


In [12]:
new_price_df.head(5)

Data = pd.merge(Data,new_price_df, on='id')

Data.head(5)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix,month,offpeak_diff_dec_january_energy_y,offpeak_diff_dec_january_power_y,peak_diff_dec_jan_energy,peak_diff_dec_jan_power
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.0,...,0.103395,0.071536,40.565969,24.339581,16.226389,1,0.020057,3.700961,-0.017912,-0.103395
1,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.0,...,0.103395,0.071536,40.565969,24.339581,16.226389,2,0.020057,3.700961,-0.017912,-0.103395
2,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.0,...,0.103395,0.071536,40.565973,24.339578,16.226383,3,0.020057,3.700961,-0.017912,-0.103395
3,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.0,...,0.103395,0.071536,40.565973,24.339578,16.226383,4,0.020057,3.700961,-0.017912,-0.103395
4,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.0,...,0.103395,0.071536,40.565973,24.339578,16.226383,5,0.020057,3.700961,-0.017912,-0.103395


In [13]:
list(price_df)

['id',
 'price_date',
 'price_off_peak_var',
 'price_peak_var',
 'price_mid_peak_var',
 'price_off_peak_fix',
 'price_peak_fix',
 'price_mid_peak_fix',
 'month']

In [14]:
### Average price across region.

mean_price = price_df.groupby(['id','price_date']).agg({'price_off_peak_var':'mean', 'price_peak_var':'mean', 'price_mid_peak_var':'mean', 'price_off_peak_fix':'mean', 'price_peak_fix':'mean', 'price_mid_peak_fix':'mean' }).reset_index()

mean_price.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,0002203ffbb812588b632b9e628cc38d,2015-01-01,0.126098,0.103975,0.070232,40.565969,24.339581,16.226389
1,0002203ffbb812588b632b9e628cc38d,2015-02-01,0.126098,0.103975,0.070232,40.565969,24.339581,16.226389
2,0002203ffbb812588b632b9e628cc38d,2015-03-01,0.128067,0.105842,0.073773,40.728885,24.437330,16.291555
3,0002203ffbb812588b632b9e628cc38d,2015-04-01,0.128067,0.105842,0.073773,40.728885,24.437330,16.291555
4,0002203ffbb812588b632b9e628cc38d,2015-05-01,0.128067,0.105842,0.073773,40.728885,24.437330,16.291555


In [15]:
### Difference in the Average price across all region.


mean_price['diff_price_off_price_peak_var_mean'] = mean_price['price_off_peak_var'] - mean_price['price_peak_var']
mean_price['diff_price_off_price_mid_var_mean'] = mean_price['price_off_peak_var'] - mean_price['price_mid_peak_var']
mean_price['diff_price_peak_price_mid_var_mean'] = mean_price['price_peak_var'] - mean_price['price_mid_peak_var']
mean_price['diff_price_off_price_peak_fix_mean'] = mean_price['price_off_peak_fix'] - mean_price['price_peak_fix']
mean_price['diff_price_off_price_mid_fix_mean'] = mean_price['price_off_peak_fix'] - mean_price['price_mid_peak_fix']
mean_price['diff_price_peak_price_mid_fix_mean'] = mean_price['price_peak_fix'] - mean_price['price_mid_peak_fix']



columns = ['id','diff_price_off_price_peak_var_mean','diff_price_off_price_mid_var_mean','diff_price_peak_price_mid_var_mean','diff_price_off_price_peak_fix_mean','diff_price_off_price_mid_fix_mean','diff_price_peak_price_mid_fix_mean']

Data = pd.merge(Data, mean_price[columns], on='id')


Data.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,offpeak_diff_dec_january_energy_y,offpeak_diff_dec_january_power_y,peak_diff_dec_jan_energy,peak_diff_dec_jan_power,diff_price_off_price_peak_var_mean,diff_price_off_price_mid_var_mean,diff_price_peak_price_mid_var_mean,diff_price_off_price_peak_fix_mean,diff_price_off_price_mid_fix_mean,diff_price_peak_price_mid_fix_mean
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.0,...,0.020057,3.700961,-0.017912,-0.103395,0.022581,0.05444,0.031859,16.226389,24.339581,8.113192
1,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.0,...,0.020057,3.700961,-0.017912,-0.103395,0.022581,0.05444,0.031859,16.226389,24.339581,8.113192
2,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.0,...,0.020057,3.700961,-0.017912,-0.103395,0.022581,0.05444,0.031859,16.226395,24.339590,8.113195


In [16]:
list(price_df)

['id',
 'price_date',
 'price_off_peak_var',
 'price_peak_var',
 'price_mid_peak_var',
 'price_off_peak_fix',
 'price_peak_fix',
 'price_mid_peak_fix',
 'month']

### Maximum Price changes across periods and Months

In [17]:
### First get the mean of all the periods


mean_prices_by_months = price_df.groupby(['id','price_date']).agg({'price_off_peak_var':'mean', 'price_peak_var':'mean', 'price_mid_peak_var':'mean', 'price_off_peak_fix':'mean', 'price_peak_fix':'mean', 'price_mid_peak_fix':'mean'}).reset_index()

In [18]:
list(mean_prices_by_months)

['id',
 'price_date',
 'price_off_peak_var',
 'price_peak_var',
 'price_mid_peak_var',
 'price_off_peak_fix',
 'price_peak_fix',
 'price_mid_peak_fix']

In [19]:
### Get the mean difference of all price periods.



mean_prices_by_months['diff_off_peak_peak_var'] = mean_prices_by_months['price_off_peak_var'] - mean_prices_by_months['price_peak_var']
mean_prices_by_months['diff_off_peak_mid_peak_var'] = mean_prices_by_months['price_off_peak_var'] - mean_prices_by_months['price_mid_peak_var']
mean_prices_by_months['diff_peak_mid_peak_var'] = mean_prices_by_months['price_peak_var'] - mean_prices_by_months['price_mid_peak_var']
mean_prices_by_months['diff_off_peak_peak_fix'] = mean_prices_by_months['price_off_peak_fix'] - mean_prices_by_months['price_peak_fix']
mean_prices_by_months['diff_off_peak_mid_peak_fix'] = mean_prices_by_months['price_off_peak_fix'] - mean_prices_by_months['price_mid_peak_fix']
mean_prices_by_months['diff_peak_mid_peak_fix'] = mean_prices_by_months['price_peak_fix'] - mean_prices_by_months['price_mid_peak_fix']


In [20]:
### Calculating the Maximum Difference across all periods.

max_diff_of_periods = mean_prices_by_months.groupby('id').agg({'diff_off_peak_peak_var':'max', 'diff_off_peak_mid_peak_var':'max', 'diff_peak_mid_peak_var':'max', 'diff_off_peak_peak_fix':'max','diff_off_peak_mid_peak_fix':'max', 'diff_peak_mid_peak_fix':'max'}).reset_index().rename(
columns={'diff_off_peak_peak_var':'diff_off_peak_peak_var_max',
        'diff_off_peak_mid_peak_var':'diff_off_peak_mid_peak_var_max',
        'diff_peak_mid_peak_var':'diff_peak_mid_peak_var_max',
        'diff_off_peak_peak_fix':'diff_off_peak_peak_fix_max',
         'diff_off_peak_mid_peak_fix':'diff_off_peak_mid_peak_fix_max',
         'diff_peak_mid_peak_fix':'diff_peak_mid_peak_fix_max'})
         



columns = ['id','diff_off_peak_peak_var_max','diff_off_peak_mid_peak_var_max','diff_peak_mid_peak_var_max','diff_off_peak_peak_fix_max','diff_off_peak_mid_peak_fix_max','diff_peak_mid_peak_fix_max']    
    

Data = pd.merge(Data,max_diff_of_periods[columns], on='id')

len(Data)

2098330

In [23]:
len(Data.columns)

70